In [1]:
import pandas as pd
import numpy as np

import os
import sys
sys.path.append(os.path.dirname(os.path.abspath(os.path.abspath(''))))
from yumspeak_ml.params import *

import geocoder

In [2]:
b1_restaurant_df = pd.read_csv('/Users/triciascy/code/scytricia/yumspeak/raw_data/batch1_restaurants_dataset.csv').drop(columns=['Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
b1_restaurant_df.head().T

,0,1,2,3,4
place_id,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,ChIJuXEcL-gX2jEReUwbpfL8C4c,ChIJebVvXAAZ2jERRlzs4R2Qsm4,ChIJLSQEyKwZ2jERsJYlxkDMr1g,ChIJLTGN8D4Z2jERP4VNFXTnIpQ
name,(Fu Chan) Harummanis Muslim Food,#01-22 Wufu Hakka Noodle,116 PHỐ ĐÊM,123 ZÔ - Ẩm Thực Việt,123 Zô The Việt Kitchen
reviews,35,1,3,252,19
main_category,Malaysian restaurant,Noodle shop,Vietnamese restaurant,Restaurant,Vietnamese restaurant
categories,['Malaysian restaurant'],['Noodle shop'],['Vietnamese restaurant'],['Restaurant'],['Vietnamese restaurant']
main_rating,2.1,5.0,5.0,4.4,3.7
address,"145 Teck Whye Ln, Singapore 680145","590 Upper Thomson Rd, #01-22, Singapore 574419","116 Mackenzie Rd, Singapore 228710","12 Gopeng St, #01-45, 49-51 icon village, Sing...","18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ..."
link,https://www.google.com/maps/place/(Fu+Chan)+Ha...,https://www.google.com/maps/place/%2301-22+Wuf...,https://www.google.com/maps/place/116+PH%E1%BB...,https://www.google.com/maps/place/123+Z%C3%94+...,https://www.google.com/maps/place/123+Z%C3%B4+...
review_photos,['https://lh5.googleusercontent.com/p/AF1QipNq...,[],['https://lh5.googleusercontent.com/p/AF1QipP1...,['https://lh5.googleusercontent.com/p/AF1QipNM...,['https://lh5.googleusercontent.com/p/AF1QipNI...


### Drop Duplicates

In [3]:
print(b1_restaurant_df.shape)
b1_restaurant_df.duplicated().sum()

(1356, 9)


173

In [4]:
b1 = b1_restaurant_df.drop_duplicates(subset=['place_id', 'name', 'reviews', 'address']).copy()
b1.shape

(1134, 9)

In [5]:
cats_to_remove = CAT_TO_REMOVE

cats_to_remove = [cat.lower() for cat in cats_to_remove]
mask = b1['main_category'].str.lower().isin(cats_to_remove)
b1_r_df = b1[~mask].copy()

In [6]:
b1_r_df.shape

(882, 9)

In [7]:
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       882 non-null    object 
 1   name           882 non-null    object 
 2   reviews        882 non-null    int64  
 3   main_category  863 non-null    object 
 4   categories     863 non-null    object 
 5   main_rating    882 non-null    float64
 6   address        873 non-null    object 
 7   link           882 non-null    object 
 8   review_photos  882 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 68.9+ KB


In [8]:
# Fill na in main_category as unknown
b1_r_df.main_category.fillna('unknown', inplace=True)
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       882 non-null    object 
 1   name           882 non-null    object 
 2   reviews        882 non-null    int64  
 3   main_category  882 non-null    object 
 4   categories     863 non-null    object 
 5   main_rating    882 non-null    float64
 6   address        873 non-null    object 
 7   link           882 non-null    object 
 8   review_photos  882 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 68.9+ KB


In [9]:
# Fill na in categories as unknown
b1_r_df['categories'] = b1_r_df['categories'].fillna("['unknown']").apply(eval)

In [10]:
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 1355
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       882 non-null    object 
 1   name           882 non-null    object 
 2   reviews        882 non-null    int64  
 3   main_category  882 non-null    object 
 4   categories     882 non-null    object 
 5   main_rating    882 non-null    float64
 6   address        873 non-null    object 
 7   link           882 non-null    object 
 8   review_photos  882 non-null    object 
dtypes: float64(1), int64(1), object(7)
memory usage: 68.9+ KB


### Generate Coordinates lat lng

In [11]:
import re

def get_coordinates(link):
    match = re.search('!3d(-?\d+(?:\.\d+)?)!4d(-?\d+(?:\.\d+))', link)
    coordinates = [float(match.group(1)), float(match.group(2))]
    return coordinates

In [12]:
b1_r_df['coordinates'] = b1_r_df['link'].apply(lambda x: get_coordinates(x))
b1_r_df.head()

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,[Malaysian restaurant],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]"
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,[Noodle shop],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]"
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,[Vietnamese restaurant],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]"
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,[Restaurant],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]"
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,[Vietnamese restaurant],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]"


In [13]:
b1_r_df['latitude'] = b1_r_df['coordinates'].apply(lambda x: x[0])
b1_r_df['longtitude'] = b1_r_df['coordinates'].apply(lambda x: x[1])
b1_r_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 1355
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       882 non-null    object 
 1   name           882 non-null    object 
 2   reviews        882 non-null    int64  
 3   main_category  882 non-null    object 
 4   categories     882 non-null    object 
 5   main_rating    882 non-null    float64
 6   address        873 non-null    object 
 7   link           882 non-null    object 
 8   review_photos  882 non-null    object 
 9   coordinates    882 non-null    object 
 10  latitude       882 non-null    float64
 11  longtitude     882 non-null    float64
dtypes: float64(3), int64(1), object(8)
memory usage: 89.6+ KB


In [14]:
def add_lat_lng(df):
    df['coordinates'] = df['link'].apply(lambda x: get_coordinates(x))
    df['latitude'] = df['coordinates'].apply(lambda x: x[0])
    df['longtitude'] = df['coordinates'].apply(lambda x: x[1])
    return df

In [15]:
test_b1 = add_lat_lng(b1_r_df)
test_b1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 882 entries, 0 to 1355
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       882 non-null    object 
 1   name           882 non-null    object 
 2   reviews        882 non-null    int64  
 3   main_category  882 non-null    object 
 4   categories     882 non-null    object 
 5   main_rating    882 non-null    float64
 6   address        873 non-null    object 
 7   link           882 non-null    object 
 8   review_photos  882 non-null    object 
 9   coordinates    882 non-null    object 
 10  latitude       882 non-null    float64
 11  longtitude     882 non-null    float64
dtypes: float64(3), int64(1), object(8)
memory usage: 89.6+ KB


In [16]:
test_b1['latitude'].sort_values(ascending=True).value_counts()

1.292072     4
1.291135     3
1.292811     2
1.304580     2
1.321491     2
            ..
1.291109     1
1.291111     1
1.291333     1
1.291374     1
52.159920    1
Name: latitude, Length: 863, dtype: int64

In [17]:
# Singapore lat lng borders
# lat 1.129 1.493
# lng 103.557 104.131

test_b1 = test_b1[test_b1['latitude'].between(left=1.129, right=1.493)]
test_b1 = test_b1[test_b1['longtitude'].between(left=103.557, right=104.131)]
test_b1 = test_b1.reset_index(drop=True)
print(test_b1.shape)
test_b1.head()

(867, 12)


,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,[Malaysian restaurant],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]",1.381337,103.752499
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,[Noodle shop],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]",1.372534,103.829059
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,[Vietnamese restaurant],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]",1.306641,103.847282
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,[Restaurant],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]",1.275403,103.844530
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,[Vietnamese restaurant],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]",1.280760,103.850414


### Get postal / district code

In [18]:
test_b1[test_b1['address'].isna()].shape

(8, 12)

In [19]:
test_b1.coordinates.isna().sum()

0

In [20]:
coor_1 = test_b1.coordinates[0]
coor_1

[1.3813371, 103.7524994]

In [21]:
postal = geocoder.mapbox(coor_1, method='reverse', key=MAP_API).json['postal'][0:2]

In [22]:
int(postal)

68

In [39]:
# get_postal from map box if address is na
test_b1['district_code'] = test_b1[test_b1['address'].isna()].coordinates.apply(lambda x: get_district_code(x))
test_b1

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,district_code
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,[Malaysian restaurant],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]",1.381337,103.752499,NaN
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,[Noodle shop],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]",1.372534,103.829059,NaN
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,[Vietnamese restaurant],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]",1.306641,103.847282,NaN
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,[Restaurant],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]",1.275403,103.844530,NaN
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,[Vietnamese restaurant],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]",1.280760,103.850414,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
862,ChIJK9FuLKEZ2jERZYDXi6XtZ2k,Coleman's Cafe,54,Cafe,[Cafe],3.7,"3 Coleman St, Singapore 179805",https://www.google.com/maps/place/Coleman's+Ca...,['https://lh5.googleusercontent.com/p/AF1QipPa...,"[1.2918776, 103.8500267]",1.291878,103.850027,NaN
863,ChIJ02kZpakZ2jERDQVTq10I8b0,Colony,2078,Restaurant,"[Restaurant, Buffet restaurant]",4.6,"7 Raffles Ave., Singapore 039799",https://www.google.com/maps/place/Colony/data=...,['https://lh5.googleusercontent.com/p/AF1QipMF...,"[1.2908195, 103.86009]",1.290820,103.860090,NaN
864,ChIJKdihA68Z2jER07IZ_I3hEVI,Colony Capital Asia Pacific Pte Ltd,0,unknown,[unknown],0.0,NaN,https://www.google.com/maps/place/Colony+Capit...,[],"[1.2956745, 103.8589973]",1.295675,103.858997,03
865,ChIJ0eEWGp8Z2jERU0GVV6u922I,Dutch Colony Coffee Co.,382,Coffee shop,[Coffee shop],4.2,"81 Clemenceau Ave, #01-21, Singapore 239917",https://www.google.com/maps/place/Dutch+Colony...,['https://lh5.googleusercontent.com/p/AF1QipMJ...,"[1.2922763, 103.8425461]",1.292276,103.842546,NaN


In [23]:
row = test_b1.iloc[[0]]
row.index

Int64Index([0], dtype='int64')

In [34]:
# address column not na, find postal code. if error find postal code with mapbox
# not na find postal code with mapbox error return address
def get_postal_code(row):
    if isinstance(row['address'], str):
        try:
            match = re.search(r'\b\d{6}\b', row['address'])
            postal_code = match.group(0)
            row['postal_code'] = postal_code[:2]
            return row
        except:
            g =geocoder.mapbox(row['coordinates'], method='reverse', key=MAP_API)
            row['address'] = g.json['postal']
            row['postal_code'] = g.json['postal'][:2]
            return row
    else:
        try:
            g =geocoder.mapbox(row['coordinates'], method='reverse', key=MAP_API)
            row['address'] = g.json['postal']
            row['postal_code'] = g.json['postal'][:2]
            return row
        except:
            return(row['address'])

In [25]:
test = test_b1.iloc[60:70]
test

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude
60,ChIJJduEcK8X2jERDKm60yWxNMY,ABC Vegetarian Food Pte Ltd,9,Vegetarian restaurant,[Vegetarian restaurant],4.3,"510 Lor 6 Toa Payoh, Singapore 319398",https://www.google.com/maps/place/ABC+Vegetari...,['https://lh5.googleusercontent.com/p/AF1QipPa...,"[1.3326718, 103.8473498]",1.332672,103.847350
61,ChIJFfDVN6UZ2jERSxUkcOpAicw,ADDA,1484,Modern Indian restaurant,[Modern Indian restaurant],4.3,"7500E Beach Rd, #01-201 Diners Building, Singa...",https://www.google.com/maps/place/ADDA/@1.3516...,['https://lh5.googleusercontent.com/p/AF1QipMf...,"[1.2996528, 103.8598725]",1.299653,103.859872
62,ChIJd3ufhzUZ2jER3EcoEcvx6Gc,AMI Patisserie,91,Patisserie,[Patisserie],4.4,"27 Scotts Rd, Singapore 228222",https://www.google.com/maps/place/AMI+Patisser...,['https://lh5.googleusercontent.com/p/AF1QipPU...,"[1.3105554, 103.8341477]",1.310555,103.834148
63,ChIJr9bboNwZ2jER9K-2k6c_bPU,AMTHUCVIETNAM,82,Vietnamese restaurant,[Vietnamese restaurant],3.6,"44 Foch Rd, Singapore 209270",https://www.google.com/maps/place/AMTHUCVIETNA...,['https://lh5.googleusercontent.com/p/AF1QipPj...,"[1.3126478, 103.8604959]",1.312648,103.860496
64,ChIJlQEbYh4Z2jERCmO7X7rN_c0,ANJU,385,Korean restaurant,"[Korean restaurant, Bistro, Izakaya restaurant]",4.3,"62 Tras St, Singapore 079001",https://www.google.com/maps/place/ANJU/@1.2777...,['https://lh5.googleusercontent.com/p/AF1QipMp...,"[1.2778061, 103.8442373]",1.277806,103.844237
65,ChIJW8WZmxIZ2jERLf6JIa4U8A8,APC (Amoy Street),0,Indonesian restaurant,"[Indonesian restaurant, Restaurant]",0.0,"87 Amoy St, Singapore 069111",https://www.google.com/maps/place/APC+(Amoy+St...,[],"[1.279361, 103.846764]",1.279361,103.846764
66,ChIJl48YNKwZ2jERObGdEz6U_a0,AR - Rahman Prata & Teh Tarik,17,Indian restaurant,[Indian restaurant],3.8,"1 Shenton Wy, One Shenton, Singapore 068803",https://www.google.com/maps/place/AR+-+Rahman+...,['https://lh5.googleusercontent.com/p/AF1QipMh...,"[1.2791066, 103.8503463]",1.279107,103.850346
67,ChIJretR-A8Z2jER8CtxRevCnLI,ART ICE,173,Dessert restaurant,[Dessert restaurant],4.4,"20 Duxton Hill, Singapore 089603",https://www.google.com/maps/place/ART+ICE/@1.2...,['https://lh5.googleusercontent.com/p/AF1QipOF...,"[1.2785478, 103.8425902]",1.278548,103.842590
68,ChIJXRdkcZcZ2jERl2gXORBm460,AS Indian Classic Cuisine,32,Indian restaurant,[Indian restaurant],3.6,NaN,https://www.google.com/maps/place/AS+Indian+Cl...,['https://lh5.googleusercontent.com/p/AF1QipPE...,"[1.2803929, 103.8506341]",1.280393,103.850634
69,ChIJQWBeXIEZ2jERpo6nf10cCd0,ASAP & Co,661,Steak house,"[Steak house, Fine dining restaurant, Halal re...",4.5,"70 Telok Ayer St, #01-01, Singapore 048458",https://www.google.com/maps/place/ASAP+%26+Co/...,['https://lh5.googleusercontent.com/p/AF1QipPd...,"[1.2829236, 103.8488431]",1.282924,103.848843


In [26]:
test = test.apply(get_postal_code, axis=1)
test

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,district_code
60,ChIJJduEcK8X2jERDKm60yWxNMY,ABC Vegetarian Food Pte Ltd,9,Vegetarian restaurant,[Vegetarian restaurant],4.3,"510 Lor 6 Toa Payoh, Singapore 319398",https://www.google.com/maps/place/ABC+Vegetari...,['https://lh5.googleusercontent.com/p/AF1QipPa...,"[1.3326718, 103.8473498]",1.332672,103.847350,31
61,ChIJFfDVN6UZ2jERSxUkcOpAicw,ADDA,1484,Modern Indian restaurant,[Modern Indian restaurant],4.3,"7500E Beach Rd, #01-201 Diners Building, Singa...",https://www.google.com/maps/place/ADDA/@1.3516...,['https://lh5.googleusercontent.com/p/AF1QipMf...,"[1.2996528, 103.8598725]",1.299653,103.859872,19
62,ChIJd3ufhzUZ2jER3EcoEcvx6Gc,AMI Patisserie,91,Patisserie,[Patisserie],4.4,"27 Scotts Rd, Singapore 228222",https://www.google.com/maps/place/AMI+Patisser...,['https://lh5.googleusercontent.com/p/AF1QipPU...,"[1.3105554, 103.8341477]",1.310555,103.834148,22
63,ChIJr9bboNwZ2jER9K-2k6c_bPU,AMTHUCVIETNAM,82,Vietnamese restaurant,[Vietnamese restaurant],3.6,"44 Foch Rd, Singapore 209270",https://www.google.com/maps/place/AMTHUCVIETNA...,['https://lh5.googleusercontent.com/p/AF1QipPj...,"[1.3126478, 103.8604959]",1.312648,103.860496,20
64,ChIJlQEbYh4Z2jERCmO7X7rN_c0,ANJU,385,Korean restaurant,"[Korean restaurant, Bistro, Izakaya restaurant]",4.3,"62 Tras St, Singapore 079001",https://www.google.com/maps/place/ANJU/@1.2777...,['https://lh5.googleusercontent.com/p/AF1QipMp...,"[1.2778061, 103.8442373]",1.277806,103.844237,07
65,ChIJW8WZmxIZ2jERLf6JIa4U8A8,APC (Amoy Street),0,Indonesian restaurant,"[Indonesian restaurant, Restaurant]",0.0,"87 Amoy St, Singapore 069111",https://www.google.com/maps/place/APC+(Amoy+St...,[],"[1.279361, 103.846764]",1.279361,103.846764,06
66,ChIJl48YNKwZ2jERObGdEz6U_a0,AR - Rahman Prata & Teh Tarik,17,Indian restaurant,[Indian restaurant],3.8,"1 Shenton Wy, One Shenton, Singapore 068803",https://www.google.com/maps/place/AR+-+Rahman+...,['https://lh5.googleusercontent.com/p/AF1QipMh...,"[1.2791066, 103.8503463]",1.279107,103.850346,06
67,ChIJretR-A8Z2jER8CtxRevCnLI,ART ICE,173,Dessert restaurant,[Dessert restaurant],4.4,"20 Duxton Hill, Singapore 089603",https://www.google.com/maps/place/ART+ICE/@1.2...,['https://lh5.googleusercontent.com/p/AF1QipOF...,"[1.2785478, 103.8425902]",1.278548,103.842590,08
68,ChIJXRdkcZcZ2jERl2gXORBm460,AS Indian Classic Cuisine,32,Indian restaurant,[Indian restaurant],3.6,048582,https://www.google.com/maps/place/AS+Indian+Cl...,['https://lh5.googleusercontent.com/p/AF1QipPE...,"[1.2803929, 103.8506341]",1.280393,103.850634,04
69,ChIJQWBeXIEZ2jERpo6nf10cCd0,ASAP & Co,661,Steak house,"[Steak house, Fine dining restaurant, Halal re...",4.5,"70 Telok Ayer St, #01-01, Singapore 048458",https://www.google.com/maps/place/ASAP+%26+Co/...,['https://lh5.googleusercontent.com/p/AF1QipPd...,"[1.2829236, 103.8488431]",1.282924,103.848843,04


In [35]:
final_b1 = test_b1.apply(get_postal_code, axis=1)
final_b1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 867 entries, 0 to 866
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   place_id       867 non-null    object 
 1   name           867 non-null    object 
 2   reviews        867 non-null    int64  
 3   main_category  867 non-null    object 
 4   categories     867 non-null    object 
 5   main_rating    867 non-null    float64
 6   address        867 non-null    object 
 7   link           867 non-null    object 
 8   review_photos  867 non-null    object 
 9   coordinates    867 non-null    object 
 10  latitude       867 non-null    float64
 11  longtitude     867 non-null    float64
 12  postal_code    867 non-null    object 
dtypes: float64(3), int64(1), object(9)
memory usage: 88.2+ KB


In [30]:
final_b1['address'].isna().sum()

0

In [31]:
final_b1['district_code'].isna().sum()

0

In [36]:
final_b1.postal_code.unique()

array(['68', '57', '22', '07', '04', '32', '06', '17', '05', '18', '10',
       '19', '31', '23', '27', '24', '73', '40', '14', '56', '76', '53',
       '52', '33', '20', '54', '16', '78', '21', '03', '08', '26', '15',
       '36', '67', '47', '38', '59', '82', '60', '25', '42', '65', '81',
       '48', '01', '12', '13', '51', '34', '39', '09', '28', '79', '46',
       '58', '55', '44', '30', '43', '45', '66', '41', '11', '64', '75',
       '80'], dtype=object)

In [ ]:
final_b1.to_csv('/Users/triciascy/code/toothyachy/yumspeak/name_data/postal_restaurant_b1.csv',index=False, header=True)

In [ ]:
#district
#region

In [ ]:
test_v1 = pd.read_csv('')

In [43]:
postal_to_district = {
    '01': '01', '02': '01', '03': '01', '04': '01', '05': '01', '06': '01',
    '07': '02', '08': '02', '09': '04', '10': '04', '11': '05', '12': '05',
    '13': '05', '14': '03', '15': '03', '16': '03', '17': '06', '18': '07',
    '19': '07', '20': '08', '21': '08', '22': '09', '23': '09', '24': '10',
    '25': '10', '26': '10', '27': '10', '28': '11', '29': '11', '30': '11',
    '31': '12', '32': '12', '33': '12', '34': '13', '35': '13', '36': '13',
    '37': '13', '38': '14', '39': '14', '40': '14', '41': '14', '42': '15',
    '43': '15', '44': '15', '45': '15', '46': '16', '47': '16', '48': '16',
    '49': '17', '50': '17', '51': '18', '52': '18', '53': '19', '54': '19',
    '55': '19', '56': '20', '57': '20', '58': '21', '59': '21', '60': '22',
    '61': '22', '62': '22', '63': '22', '64': '22', '65': '23', '66': '23',
    '67': '23', '68': '23', '69': '24', '70': '24', '71': '24', '72': '25',
    '73': '25', '75': '27', '76': '27', '77': '26', '78': '26', '79': '28',
    '80': '28', '81': '17', '82': '19'
}

In [ ]:
district_to_region = {
    '01': 'City', '02': 'City', '03': 'South', '04': 'South', '05': 'West', '06': 'City',
    '07': 'City', '08': 'Central', '09': 'Central', '10': 'Central', '11': 'Central', '12': 'Central',
    '13': 'East', '14': 'East', '15': 'East', '16': 'East', '17': 'East', '18': 'East',
    '19': 'North', '20': 'North', '21': 'West', '22': 'West', '23': 'West', '24': 'West',
    '25': 'North', '26': 'North', '27': 'North', '28': 'North'
}

In [42]:
final_b1['district_code'] = final_b1['postal_code'].map(postal_to_district)
final_b1.head()

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,postal_code,district_code
0,ChIJ8Y3ZDL8R2jERZdF0_9U5XSo,(Fu Chan) Harummanis Muslim Food,35,Malaysian restaurant,[Malaysian restaurant],2.1,"145 Teck Whye Ln, Singapore 680145",https://www.google.com/maps/place/(Fu+Chan)+Ha...,['https://lh5.googleusercontent.com/p/AF1QipNq...,"[1.3813371, 103.7524994]",1.381337,103.752499,68,West
1,ChIJuXEcL-gX2jEReUwbpfL8C4c,#01-22 Wufu Hakka Noodle,1,Noodle shop,[Noodle shop],5.0,"590 Upper Thomson Rd, #01-22, Singapore 574419",https://www.google.com/maps/place/%2301-22+Wuf...,[],"[1.3725339, 103.8290593]",1.372534,103.829059,57,North
2,ChIJebVvXAAZ2jERRlzs4R2Qsm4,116 PHỐ ĐÊM,3,Vietnamese restaurant,[Vietnamese restaurant],5.0,"116 Mackenzie Rd, Singapore 228710",https://www.google.com/maps/place/116+PH%E1%BB...,['https://lh5.googleusercontent.com/p/AF1QipP1...,"[1.3066408, 103.8472818]",1.306641,103.847282,22,Central
3,ChIJLSQEyKwZ2jERsJYlxkDMr1g,123 ZÔ - Ẩm Thực Việt,252,Restaurant,[Restaurant],4.4,"12 Gopeng St, #01-45, 49-51 icon village, Sing...",https://www.google.com/maps/place/123+Z%C3%94+...,['https://lh5.googleusercontent.com/p/AF1QipNM...,"[1.2754032, 103.8445304]",1.275403,103.844530,07,City
4,ChIJLTGN8D4Z2jERP4VNFXTnIpQ,123 Zô The Việt Kitchen,19,Vietnamese restaurant,[Vietnamese restaurant],3.7,"18 Raffles Quay, #01-48 Lau Pa Sat, Singapore ...",https://www.google.com/maps/place/123+Z%C3%B4+...,['https://lh5.googleusercontent.com/p/AF1QipNI...,"[1.2807598, 103.8504144]",1.280760,103.850414,04,City
